In [88]:
import pandas as pd
import numpy as np
import collections
import json
import re
from tqdm import tqdm

tqdm.pandas()

In [112]:
def ascii_converter(text):
    try:
        patterns = [
            (r"â\x80\x93", " to "),
            (r"\xa0", " ")
        ]
        for (pattern, replacement) in patterns:
            text = re.sub(pattern, replacement, text)
    
    except: text = text

    return text

In [98]:
root = "../../../data/interim/"

df_bert = pd.read_csv(root + "DF_GoogleForms_BERT_AllData.csv", 
                        index_col=0)
df_jacc = pd.read_csv(root + "DF_GoogleForms_Jaccard_AllData.csv",
                        index_col=0)

df_jacc["SIM"] = "Jacc"
df_bert["SIM"] = "Bert"

In [99]:
df_bert_sample = df_bert[df_bert["Sim Sum"] > 0.90]\
    # .sample(n=20, replace=False, random_state=333)

df_jacc_sample = df_jacc[df_jacc["Jaccard Sum"] > 0.0250]\
    # .sample(n=20, replace=False, random_state=333)

In [101]:
df = pd.concat(
    [df_bert_sample, df_jacc_sample]
)

In [102]:
df

,Species,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5,GoogleSentence,Sim Sum,SIM,Jaccard Sum
8,Acacia amythethophylla,leaf shape,leaf shape,['Bipinnate leaf'],"Leaves large , pinnae 7-14 pairs.","Leaves large, 10-20 cm.","Leaves large, 10â20 cm.","Pinnae sometimes 20 or more pairs, with 50 or ...","Inflorescence large, terminal, much-branched, ...",Leaf shape: bipinnate leaf,0.936336,Bert,NaN
11,Acacia amythethophylla,leaf base,leaf base,"['Leaf base attenuated', 'Rounded leaf base']","Leaves large , pinnae 7-14 pairs.","Pinnae sometimes 20 or more pairs, with 50 or ...","Leaves large, 10â20 cm.","Leaves large, 10-20 cm.","Stipules spinescent, stout, brown, glossy, com...",Leaf base: leaf base attenuated,0.930424,Bert,NaN
12,Acacia amythethophylla,leaf apex,leaf apex,"['Leaf apex sharp corner', 'Leaf apex with wid...","Pinnae sometimes 20 or more pairs, with 50 or ...","Leaves large , pinnae 7-14 pairs.","Leaves large, 10â20 cm.","Leaves large, 10-20 cm.","Stipules spinescent, stout, brown, glossy, com...",Leaf apex: leaf apex sharp corner,0.907039,Bert,NaN
18,Acacia amythethophylla,inflorescences,inflorescences,"['Glomerule of flowers or inflorescences', 'Pa...","Inflorescence large, terminal, much-branched, ...","Flowers orange or yellow, strongly and sweetly...",Shrub or tree to 7 m. Thorns in pairs at nodes...,"Flowers orange or yellow, strongly and sweetly...",Flower-heads abundant golden yellow.,Inflorescences: glomerule of flowers or inflor...,0.925787,Bert,NaN
21,Acacia amythethophylla,flower shape,flower shape,"['Five-petalled flower', 'Tubular flower']",Flower-heads abundant golden yellow.,"Inflorescence large, terminal, much-branched, ...","Flowers orange or yellow, strongly and sweetly...","Leaves large, 10â20 cm.","Leaves large, 10-20 cm.",Flower shape: five-petalled flower,0.920279,Bert,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13128,Wodyetia bifurcata,Fruit Colour Description,Fruit Colour Description,['red'],Mature red fruit on Wodyetia.,The fruit of the foxtail palm tree is green wh...,Palm Fruits: Red when ripe.,Fruit showing remains of style.,Fruit on ground below Foxtail.,Fruit colour description: red,NaN,Jacc,0.039286
13129,Wodyetia bifurcata,Fruit Colour,Fruit Colour,['red'],Mature red fruit on Wodyetia.,Palm Fruits: Red when ripe.,Fruit showing remains of style.,Fruit on ground below Foxtail.,The fruit of the foxtail palm tree is green wh...,Fruit colour: red,NaN,Jacc,0.059683
13130,Wodyetia bifurcata,Crown,Crown,['canopy'],"Thin Trunk, Crown Shaft, Medium Height,.",Note un-opened flowers below crown shaft.,Note medium sized trunk and crown shaft.,The average width of the crown of leaves on a ...,"This is a thin trunked, crown shafted, medium ...",Crown: canopy,NaN,Jacc,0.060162
13131,Wodyetia bifurcata,Fruit Size,Fruit Size,['large'],Mature red fruit on Wodyetia.,Fruit showing remains of style.,Fruit on ground below Foxtail.,Foxtail Palm Tree: Large Multi Stem.,The size of the fruit isÂ around 4 to 5 cms l...,Fruit size: large,NaN,Jacc,0.044748


In [147]:
df.loc[:, ["1", "2", "3", "4", "5"]] = df[["1", "2", "3", "4", "5"]].apply(ascii_converter)

In [172]:
def ascii_converter(text):

    if type(text) != str:
        return text

    patterns = [
        (r"â\x80\x93", " to "),
        (r"\xa0", " "),
        (r"Â", ""),
        (r"\s,", ","),  
        (r"\s+", " ")
    ]
    for (pattern, replacement) in patterns:
        text = re.sub(pattern, replacement, text)
    
    return text

In [173]:
# Note to self: Could be done using Pandas
columns = [
    "1", "2", "3", "4", "5"
]

for column in columns:
    df.loc[:, column] = df[column].apply(ascii_converter)

In [174]:
df

,Species,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5,GoogleSentence,Sim Sum,SIM,Jaccard Sum
8,Acacia amythethophylla,leaf shape,leaf shape,['Bipinnate leaf'],"Leaves large, pinnae 7-14 pairs.","Leaves large, 10-20 cm.","Leaves large, 10 to 20 cm.","Pinnae sometimes 20 or more pairs, with 50 or ...","Inflorescence large, terminal, much-branched, ...",Leaf shape: bipinnate leaf,0.936336,Bert,NaN
11,Acacia amythethophylla,leaf base,leaf base,"['Leaf base attenuated', 'Rounded leaf base']","Leaves large, pinnae 7-14 pairs.","Pinnae sometimes 20 or more pairs, with 50 or ...","Leaves large, 10 to 20 cm.","Leaves large, 10-20 cm.","Stipules spinescent, stout, brown, glossy, com...",Leaf base: leaf base attenuated,0.930424,Bert,NaN
12,Acacia amythethophylla,leaf apex,leaf apex,"['Leaf apex sharp corner', 'Leaf apex with wid...","Pinnae sometimes 20 or more pairs, with 50 or ...","Leaves large, pinnae 7-14 pairs.","Leaves large, 10 to 20 cm.","Leaves large, 10-20 cm.","Stipules spinescent, stout, brown, glossy, com...",Leaf apex: leaf apex sharp corner,0.907039,Bert,NaN
18,Acacia amythethophylla,inflorescences,inflorescences,"['Glomerule of flowers or inflorescences', 'Pa...","Inflorescence large, terminal, much-branched, ...","Flowers orange or yellow, strongly and sweetly...",Shrub or tree to 7 m. Thorns in pairs at nodes...,"Flowers orange or yellow, strongly and sweetly...",Flower-heads abundant golden yellow.,Inflorescences: glomerule of flowers or inflor...,0.925787,Bert,NaN
21,Acacia amythethophylla,flower shape,flower shape,"['Five-petalled flower', 'Tubular flower']",Flower-heads abundant golden yellow.,"Inflorescence large, terminal, much-branched, ...","Flowers orange or yellow, strongly and sweetly...","Leaves large, 10 to 20 cm.","Leaves large, 10-20 cm.",Flower shape: five-petalled flower,0.920279,Bert,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13128,Wodyetia bifurcata,Fruit Colour Description,Fruit Colour Description,['red'],Mature red fruit on Wodyetia.,The fruit of the foxtail palm tree is green wh...,Palm Fruits: Red when ripe.,Fruit showing remains of style.,Fruit on ground below Foxtail.,Fruit colour description: red,NaN,Jacc,0.039286
13129,Wodyetia bifurcata,Fruit Colour,Fruit Colour,['red'],Mature red fruit on Wodyetia.,Palm Fruits: Red when ripe.,Fruit showing remains of style.,Fruit on ground below Foxtail.,The fruit of the foxtail palm tree is green wh...,Fruit colour: red,NaN,Jacc,0.059683
13130,Wodyetia bifurcata,Crown,Crown,['canopy'],"Thin Trunk, Crown Shaft, Medium Height,.",Note un-opened flowers below crown shaft.,Note medium sized trunk and crown shaft.,The average width of the crown of leaves on a ...,"This is a thin trunked, crown shafted, medium ...",Crown: canopy,NaN,Jacc,0.060162
13131,Wodyetia bifurcata,Fruit Size,Fruit Size,['large'],Mature red fruit on Wodyetia.,Fruit showing remains of style.,Fruit on ground below Foxtail.,Foxtail Palm Tree: Large Multi Stem.,The size of the fruit is around 4 to 5 cms len...,Fruit size: large,NaN,Jacc,0.044748


In [175]:
list(df.loc[13131, "5"])[1]

'The size of the fruit is around 4 to 5 cms lenght & 2- 3 cm breadth.'